In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U -q "tf-models-official"
!pip install -U nvidia-cudnn-cu11

# Install the mediapy package for visualizing images/videos.
# See https://github.com/google/mediapy
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy remotezip
!pip install -U -q git+https://github.com/tensorflow/docs

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.9.1 requires tensorflow~=2.16.1, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.7/566.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.4 MB/s eta 0:00:00


In [3]:
!pip install tf-models-official  --quiet
!pip install tensorflow-docs --quiet
!pip install -U kaleido --quiet
!pip install tf-keras-vis --quiet

In [4]:
import random
import pathlib
import itertools
import collections
from collections import defaultdict

import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import imageio
import gc

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard, LearningRateScheduler

2024-11-28 11:07:59.013487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732792079.035998      23 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732792079.043029      23 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

In [6]:
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model
from official.projects.movinet.tools import export_saved_model

In [7]:
def classes_available(path):
    return [c for c in os.listdir(path)]

def files_per_class(path):
    class_instances = dict()
    for c in classes_available(path):
        class_path = os.path.join(path,c)
        files = list()
        for file in os.listdir(class_path):
            files.append(os.path.join(class_path,file))
        class_instances[c.split('-')[0]] = files

    return class_instances

t = files_per_class('/kaggle/input/footballactions/FootballActions')

print('Num classes:', len(list(t.keys())))
for i in t.keys():
    print(f'Num videos for class {i}:' ,len(t[i]))


def average_frames_and_length_per_class(path):
    class_instances = files_per_class(path)
    avg_frames_per_class = {}
    avg_length_per_class = {}

    for class_name, files in class_instances.items():
        total_frames = 0
        total_length = 0
        for file in files:
            cap = cv2.VideoCapture(file)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            duration = frame_count / fps if fps > 0 else 0

            total_frames += frame_count
            total_length += duration
            cap.release()

        num_files = len(files)
        avg_frames = total_frames / num_files if num_files else 0
        avg_length = total_length / num_files if num_files else 0

        avg_frames_per_class[class_name] = avg_frames
        avg_length_per_class[class_name] = avg_length

    return avg_frames_per_class, avg_length_per_class


avg_frames_per_class, avg_length_per_class = average_frames_and_length_per_class('/kaggle/input/footballactions/FootballActions')

print('\nAverage frames and length per class:')
for class_name in avg_frames_per_class.keys():
    print(f'{class_name}:')
    print(f'  Avg frames: {avg_frames_per_class[class_name]:.2f}')
    print(f'  Avg length: {avg_length_per_class[class_name]:.2f} seconds')

def train_test_split(dataset_dict, test_split=0.2, seed=42):
    random.seed(seed)
    train_dict = dict()
    test_dict = dict()

    for class_name, files in dataset_dict.items():
        random.shuffle(files)
        split_index = int(len(files) * (1 - test_split))
        train_files = files[:split_index]
        test_files = files[split_index:]
        train_dict[class_name] = train_files
        test_dict[class_name] = test_files

    return train_dict, test_dict

Num classes: 8
Num videos for class shortpass: 354
Num videos for class longpass: 147
Num videos for class throw: 367
Num videos for class goalkick: 425
Num videos for class penalty: 322
Num videos for class corner: 411
Num videos for class freekick: 374
Num videos for class ontarget: 340

Average frames and length per class:
shortpass:
  Avg frames: 75.86
  Avg length: 2.99 seconds
longpass:
  Avg frames: 101.80
  Avg length: 4.07 seconds
throw:
  Avg frames: 62.64
  Avg length: 2.46 seconds
goalkick:
  Avg frames: 73.02
  Avg length: 2.88 seconds
penalty:
  Avg frames: 71.91
  Avg length: 2.86 seconds
corner:
  Avg frames: 75.38
  Avg length: 2.89 seconds
freekick:
  Avg frames: 74.21
  Avg length: 2.88 seconds
ontarget:
  Avg frames: 69.32
  Avg length: 2.48 seconds


In [8]:
from sklearn.utils.class_weight import compute_class_weight

def compute_inverse_class_weights(dataset_dict):
    """
    Compute class weights using inverse frequency.
    Higher weight for classes with fewer samples.
    """
    # Get sample counts for each class
    class_counts = {class_name: len(files) for class_name, files in dataset_dict.items()}
    total_samples = sum(class_counts.values())
    
    # Compute weights inversely proportional to class frequencies
    class_weights = {}
    for class_name, count in class_counts.items():
        class_weights[class_name] = total_samples / (len(class_counts) * count)
    
    return class_weights

def compute_balanced_class_weights(dataset_dict):
    """
    Compute class weights using sklearn's balanced weighting.
    This generally provides better balance than simple inverse frequency.
    """
    # Prepare data for sklearn's compute_class_weight
    classes = list(dataset_dict.keys())
    y = []
    for class_name, files in dataset_dict.items():
        y.extend([class_name] * len(files))
    
    # Compute balanced weights
    weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y),
        y=y
    )
    
    return dict(zip(classes, weights))

def compute_effective_samples_weights(dataset_dict, beta=0.999):
    """
    Compute class weights using effective number of samples.
    This method helps prevent the model from being too aggressive 
    in favoring minority classes.
    
    Reference: "Class-Balanced Loss Based on Effective Number of Samples"
    """
    # Get sample counts for each class
    class_counts = {class_name: len(files) for class_name, files in dataset_dict.items()}
    
    # Compute effective number of samples for each class
    class_weights = {}
    for class_name, n in class_counts.items():
        # Calculate effective number using the paper's formula
        effective_num = (1.0 - beta ** n) / (1.0 - beta)
        class_weights[class_name] = (1 - beta) / (1 - beta ** n)
    
    # Normalize weights
    total_weight = sum(class_weights.values())
    for class_name in class_weights:
        class_weights[class_name] *= len(class_weights) / total_weight
    
    return class_weights


In [9]:
# Compute weights using different methods
inverse_weights = compute_inverse_class_weights(t)
balanced_weights = compute_balanced_class_weights(t)
effective_weights = compute_effective_samples_weights(t)

# Create class index mapping
class_indices = {name: idx for idx, name in enumerate(t.keys())}

# Convert to index-based weights for all methods
index_inverse_weights = {class_indices[k]: v for k, v in inverse_weights.items()}
index_balanced_weights = {class_indices[k]: v for k, v in balanced_weights.items()}
index_effective_weights = {class_indices[k]: v for k, v in effective_weights.items()}

In [10]:
# Print results for comparison
print("Class distribution:")
for class_name, files in t.items():
    print(f"{class_name}: {len(files)} samples")

print("\nInverse frequency weights:")
for idx, weight in sorted(index_inverse_weights.items()):
    print(f"Class {idx}: {weight:.4f}")

print("\nBalanced weights (sklearn):")
for idx, weight in sorted(index_balanced_weights.items()):
    print(f"Class {idx}: {weight:.4f}")

print("\nEffective samples weights:")
for idx, weight in sorted(index_effective_weights.items()):
    print(f"Class {idx}: {weight:.4f}")

Class distribution:
shortpass: 354 samples
longpass: 147 samples
throw: 367 samples
goalkick: 425 samples
penalty: 322 samples
corner: 411 samples
freekick: 374 samples
ontarget: 340 samples

Inverse frequency weights:
Class 0: 0.9675
Class 1: 2.3299
Class 2: 0.9332
Class 3: 0.8059
Class 4: 1.0637
Class 5: 0.8333
Class 6: 0.9158
Class 7: 1.0074

Balanced weights (sklearn):
Class 0: 0.8333
Class 1: 0.9158
Class 2: 0.8059
Class 3: 2.3299
Class 4: 1.0074
Class 5: 1.0637
Class 6: 0.9675
Class 7: 0.9332

Effective samples weights:
Class 0: 0.8919
Class 1: 1.9450
Class 2: 0.8656
Class 3: 0.7680
Class 4: 0.9658
Class 5: 0.7890
Class 6: 0.8522
Class 7: 0.9225


In [11]:
def get_balanced_class_weights(train_dict):
    """
    Compute balanced class weights using sklearn's implementation.
    
    Args:
        train_dict: Dictionary mapping class names to lists of training samples
    Returns:
        Dictionary mapping class indices to their balanced weights
    """
    # Create labels array for compute_class_weight
    labels = []
    label_to_index = {}
    for idx, class_name in enumerate(train_dict.keys()):
        label_to_index[class_name] = idx
        labels.extend([idx] * len(train_dict[class_name]))
    
    # Compute balanced weights
    classes = np.unique(labels)
    weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=labels
    )
    
    # Create dictionary mapping class indices to weights
    class_weights = dict(zip(range(len(weights)), weights))
    
    return class_weights

In [12]:
class_weights = get_balanced_class_weights(t)
print("\nUsing balanced class weights:")
for idx, weight in sorted(class_weights.items()):
    print(f"Class {idx}: {weight:.4f}")


Using balanced class weights:
Class 0: 0.9675
Class 1: 2.3299
Class 2: 0.9332
Class 3: 0.8059
Class 4: 1.0637
Class 5: 0.8333
Class 6: 0.9158
Class 7: 1.0074


In [13]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

In [14]:
def per_model(t, train_dict, test_dict, model_class, split):
    split_percentage = 100 * split
    classes = list(t.keys())
    train_counts = [len(train_dict[c]) for c in classes]
    test_counts = [len(test_dict[c]) for c in classes]
    
    # Sort classes by total count
    sorted_data = sorted(zip(classes, train_counts, test_counts), key=lambda x: x[1] + x[2], reverse=True)
    sorted_classes, sorted_train_counts, sorted_test_counts = zip(*sorted_data)

    fig = go.Figure()

    # Add bars for train and test counts
    fig.add_trace(go.Bar(
        x=sorted_classes,
        y=sorted_train_counts,
        name='Train',
        marker_color='#9370DB',  # Medium purple color
        text=[f'{count:,} ({count/(count+test_count):.1%})' for count, test_count in zip(sorted_train_counts, sorted_test_counts)],
        textposition='auto',
        hoverinfo='text+name'
    ))
    fig.add_trace(go.Bar(
        x=sorted_classes,
        y=sorted_test_counts,
        name='Test',
        marker_color='#D3D3D3',  # Light gray color
        text=[f'{count:,} ({count/(count+train_count):.1%})' for count, train_count in zip(sorted_test_counts, sorted_train_counts)],
        textposition='auto',
        hoverinfo='text+name'
    ))

    # Calculate overall statistics
    total_train = sum(train_counts)
    total_test = sum(test_counts)
    total_overall = total_train + total_test

    # Update layout
    fig.update_layout(
        title={
            'text': f'Training-Test Split for {model_class}',
            'font': {'family': 'Segoe Print', 'size': 24},
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title={'text': 'Class', 'font': {'family': 'Segoe Print', 'size': 18}},
        yaxis_title={'text': 'Number of Videos', 'font': {'family': 'Segoe Print', 'size': 18}},
        annotations=[
            dict(
                x=0.5, y=1.05,
                showarrow=False,
                text=f'Total Videos: {total_overall:,} | Train: {total_train:,} ({total_train/total_overall:.1%}) | Test: {total_test:,} ({total_test/total_overall:.1%})',
                xref='paper', yref='paper',
                font=dict(family='Segoe Print', size=16, color='darkred')
            )
        ],
        hovermode='closest',
        margin=dict(t=150),  # Increase top margin to accommodate the title and annotation
        font=dict(family='Segoe Print'),  # Set global font
        plot_bgcolor='#F8F8F8',  # Very light grey background
        paper_bgcolor='#F8F8F8'  # Match the plot background color
    )

    # Update axes
    fig.update_xaxes(tickangle=45, tickfont=dict(family='Segoe Print', size=12))
    fig.update_yaxes(gridcolor='white', griddash='dash')  # Changed grid color to white for better contrast

    # Add secondary y-axis for percentages
    fig.update_layout(
        yaxis2=dict(
            title='Percentage',
            titlefont=dict(family='Segoe Print', size=18),
            overlaying='y',
            side='right',
            showgrid=False,
            range=[0, 100],
            ticksuffix='%'
        )
    )

    # Save the figure
    fig.write_image(f'detailed_train_test_split_distribution_for_{model_class}_at_training_split_{100-split_percentage:.0f}_{split_percentage:.0f}.png', scale=3, width=1200, height=800)
    print(f"Train-Test Split Distribution plot saved for {model_class} with {split_percentage:.1f}% split.")

In [15]:
def format_frames(frame, output_size):
    """
    Formats video frames to specified size.
    """
    frame = cv2.resize(frame, output_size)
    frame = frame / 255.0
    return frame

def create_blank_frames(n_frames, output_size):
    """
    Creates blank frames for error cases.
    """
    return np.zeros((n_frames, output_size[0], output_size[1], 3), dtype=np.float32)

In [16]:

def frames_from_video_file_optimized(video_path, n_frames=8, output_size=(172,172)):
    """
    Statistics-aware frame extraction optimized for memory constraints.
    
    Args:
        video_path: Path to video file
        
        n_frames: Number of frames to extract
        output_size: Output frame dimensions
    
    Returns:
        numpy array of frames: shape (n_frames, height, width, channels)
    """
    src = cv2.VideoCapture(str(video_path))
    total_frames = int(src.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = src.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps if fps > 0 else 0
    
    # Ensure we have at least n_frames frames to sample
    if total_frames < n_frames:
        src.release()
        return create_blank_frames(n_frames, output_size)
    
    frame_step = max(total_frames // (n_frames + 1), 1)
    
    # Determine sampling strategy based on video duration
    if duration > 5.0:  # Long videos
        # Sample evenly with focus on middle section
        mid_point = total_frames // 2
        quarter = total_frames // 4
        frame_positions = [
            quarter,                        # First quarter
            mid_point - frame_step,         # Pre-middle
            mid_point,                      # Middle
            mid_point + frame_step,         # Post-middle
            3 * quarter,                    # Third quarter
            total_frames - 2*frame_step,    # Near end
            total_frames - frame_step,      # Pre-end
            total_frames - 1                # End
        ]
    elif duration > 3.0:  # Medium length videos
        # Balanced sampling
        frame_positions = np.linspace(0, total_frames-1, n_frames, dtype=int)
    else:  # Short videos
        # Dense sampling around the middle
        mid_point = total_frames // 2
        offset = frame_step // 2
        frame_positions = [
            0,                              # Start
            mid_point - 2*offset,           # Pre-action
            mid_point - offset,             # Action build-up
            mid_point,                      # Peak action
            mid_point + offset,             # Post-peak
            mid_point + 2*offset,           # Action follow-through
            total_frames - frame_step,      # Pre-end
            total_frames - 1                # End
        ]
    
    result = []
    for pos in frame_positions:
        src.set(cv2.CAP_PROP_POS_FRAMES, min(pos, total_frames - 1))
        ret, frame = src.read()
        if ret:
            frame = format_frames(frame, output_size)
            result.append(frame)
        else:
            result.append(np.zeros((output_size[0], output_size[1], 3), dtype=np.float32))
    
    src.release()
    result = np.array(result)[..., [2, 1, 0]]  # BGR to RGB
    return result

In [17]:

class FrameGenerator:
    def __init__(self, dataset_dict, num_frames, size=290, training=False):
        self.dataset_dict = dataset_dict
        self.num_frames = num_frames
        self.size = size
        self.training = training
        self.class_names = sorted(dataset_dict.keys())
        self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))
    
    def __call__(self):
        video_paths = []
        classes = []
        for class_name, files in self.dataset_dict.items():
            video_paths.extend(files)
            classes.extend([class_name] * len(files))
        
        pairs = list(zip(video_paths, classes))
        if self.training:
            random.shuffle(pairs)
            
        for video_path, class_name in pairs:
            try:
                video_frames = frames_from_video_file_optimized(video_path, 
                                                   self.num_frames, 
                                                   (self.size, self.size))
                label = self.class_ids_for_name[class_name]
                yield video_frames, label
            except Exception as e:
                print(f"Warning: Error processing video {video_path}: {str(e)}")
                yield create_blank_frames(self.num_frames, (self.size, self.size)), 0

In [18]:
models = {
#         'A0': {'batch_size': 10, 'num_frames': 8, 'resolution': 172,'split':0.30}
        'A3': {'batch_size': 8, 'num_frames': 8, 'resolution': 256,'split':0.25}
        # 'A2': {'batch_size': 10, 'num_frames': 8, 'resolution': 224,'split':0.30}
       
}

In [19]:
def prepare_dataset(generator,batch_size):
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int16)
        )
    )
    return dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [21]:

class CustomLearningRateLogger(tf.keras.callbacks.Callback):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False
    
    def on_epoch_begin(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        if hasattr(lr, 'value'):
            lr = lr.value()
        tf.summary.scalar('learning_rate', data=lr, step=epoch)
        if logs is None:
            logs = {}
        logs['lr'] = lr
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = logs.get('lr', None)

class CustomModelCheckpoint(ModelCheckpoint):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

class CustomReduceLROnPlateau(ReduceLROnPlateau):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = self.model.optimizer.lr.numpy()
        super().on_epoch_end(epoch, logs)

class CustomEarlyStopping(EarlyStopping):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

class CustomLearningRateScheduler(LearningRateScheduler):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

class CustomTensorBoard(TensorBoard):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

In [22]:
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

In [23]:
from sklearn.metrics import confusion_matrix, classification_report

def get_actual_predicted_labels(dataset, model):
    actual = []
    predicted = []
    for images, labels in dataset:
        actual.extend(labels.numpy())
        preds = model.predict(images)
        predicted.extend(tf.argmax(preds, axis=1).numpy())
    return np.array(actual), np.array(predicted)

In [24]:
from matplotlib.colors import LinearSegmentedColormap

def plot_confusion_matrix(actual, predicted, labels, ds_type, model_class, accuracy, split, normalize=False):
    split = split * 100
    
    def create_cm_plot(ax, cm, cmap, title):
        sns.heatmap(cm, annot=True, fmt=fmt, cmap=cmap,
                    xticklabels=labels, yticklabels=labels,
                    annot_kws={"size": 10, "weight": "bold"}, square=True, 
                    cbar_kws={'shrink': .8}, ax=ax, linewidths=0.5, linecolor='gray')
        
        ax.set_title(title, fontsize=18, fontweight='bold', pad=20)
        ax.set_xlabel('Predicted Action', fontsize=14, fontweight='bold', labelpad=15)
        ax.set_ylabel('Actual Action', fontsize=14, fontweight='bold', labelpad=15)
        
        # Improve tick label visibility
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=10, fontweight='bold')
        plt.setp(ax.get_yticklabels(), rotation=0, ha="right", fontsize=10, fontweight='bold')
        
        # Add value annotations
        thresh = cm.max() / 2
        for i, j in np.ndindex(cm.shape):
            ax.text(j + 0.5, i + 0.5, f"{cm[i, j]:.0f}" if not normalize else f"{cm[i, j]:.2f}",
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black",
                    fontweight='bold', fontsize=10)

        # Add borders to the heatmap
        for _, spine in ax.spines.items():
            spine.set_visible(True)
            spine.set_linewidth(2)
            spine.set_color('black')

    try:
        actual = np.ravel(actual)
        predicted = np.ravel(predicted)
        cm = confusion_matrix(actual, predicted)
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            fmt = '.2f'
        else:
            fmt = 'd'
        
        fig, (ax_cm, ax_guide) = plt.subplots(1, 2, figsize=(24, 12), 
                                              gridspec_kw={'width_ratios': [1.5, 1]})
        
        # Custom colormap for better visibility
        cmap = sns.color_palette("Blues", as_cmap=True)
        create_cm_plot(ax_cm, cm, cmap, 'Confusion Matrix')
        
        # Add interpretation guide
        ax_guide.axis('off')
        guide_text = """
        Interpretation Guide:
        
        1. Diagonal Elements: 
           Represent correct predictions. Higher values are better.
        
        2. Off-Diagonal Elements: 
           Show misclassifications. Lower values are better.
        
        3. Row Sums: 
           Total actual instances for each class.
        
        4. Column Sums: 
           Total predicted instances for each class.
        
        5. Color Intensity: 
           Darker blues indicate higher values.
        
        6. Normalized vs. Raw: 
           - Raw: Shows actual counts
           - Normalized: Shows proportions (sum to 1 for each row)
        
        7. Common Patterns:
           - Strong Diagonal: Good overall performance
           - Weak Diagonal: Poor overall performance
           - Dark Off-Diagonal: Common misclassification
        
        8. Class Imbalance: 
           Look for significantly different row sums.
        
        9. Precision vs. Recall:
           - Precision: Look down columns (few false positives)
           - Recall: Look across rows (few false negatives)
        
        10. Overall Accuracy:
            Sum of diagonal elements divided by total predictions.
        """
        ax_guide.text(0, 1, guide_text, va='top', ha='left', fontsize=12, 
                      bbox=dict(boxstyle='round,pad=0.5', facecolor='wheat', alpha=0.3),
                      fontweight='medium')
        
        plt.suptitle(f'Confusion Matrix: MoViNet-{model_class}-Stream ({ds_type})\n'
                     f'Accuracy: {accuracy*100:.4f}%', fontsize=22, fontweight='bold', y=1.02)
        
        plt.tight_layout()
        fname = f'confusion_matrix_{model_class}_{ds_type}_accuracy_{accuracy*100:.4f}.png'
        plt.savefig(fname, bbox_inches='tight', dpi=600)
        plt.close()
        print(f"Enhanced confusion matrix saved as {fname}")
        
        # Generate classification report
        report = classification_report(actual, predicted, target_names=labels, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_fname = f'classification_report_{model_class}_{ds_type}_at_training_split_{100-split}_{split}.csv'
        report_df.to_csv(report_fname)
        print(f"Classification report saved as {report_fname}")
        
    except Exception as e:
        print(f"Error in plotting confusion matrix: {str(e)}")
        import traceback
        traceback.print_exc()

In [25]:
def plot_confusion_matrix2(actual, predicted, labels, ds_type,model_class,accuracy,split):
  cm = tf.math.confusion_matrix(actual, predicted)
  split = 100 * split
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(6, 16)})
  sns.set(font_scale=1.4)
  plt.title(f'Confusion Matrix: MoViNet-{model_class}-Stream ({ds_type})\n'
                  f'Accuracy: {accuracy*100:.4f}',
                  fontsize=16, pad=16)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)
  plt.tight_layout()
  fname = f'confusion_matrix_alternative_presentation_{model_class}_{ds_type}_accuracy_{accuracy*100:.4f}%_at_training_split_{100-split}_{split}.png'
  plt.savefig(fname, bbox_inches='tight', dpi=300)
  plt.close()

In [26]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def plot_training_metrics(results, model_id, initial_lr, batch_size, optimizer, resolution, split, accuracy):
    """
    Plot training and validation metrics (accuracy, loss, and learning rate) over epochs with improved visuals for print.
    
    Args:
    results (keras.callbacks.History): History object returned by model.fit().
    model_id (str): Identifier for the model.
    initial_lr (float): Initial learning rate.
    batch_size (int): Batch size used for training.
    optimizer (str): Name of the optimizer used.
    resolution (int): Input resolution for the model.
    split (float): Train-validation split ratio.
    accuracy (float): Test accuracy of the model.
    
    Returns:
    None
    """
    history = results.history  # Extract the history dictionary from the History object
    split = 100 * split
    plt.style.use('seaborn')
    fig = plt.figure(figsize=(20, 30))
    gs = GridSpec(4, 2, figure=fig, height_ratios=[1, 1, 1, 0.5])
    
    # Colors
    colors = {'train': '#C71585',  # Deep pink
              'val': '#696969',    # Dim gray
              'lr': '#FFA500'}     # Orange
    
    # Metrics
    metrics = ['accuracy', 'loss']
    
    for i, metric in enumerate(metrics):
        ax = fig.add_subplot(gs[i, :])
        epochs = range(1, len(history[metric]) + 1)
        
        # Training data
        ax.plot(epochs, history[metric], color=colors['train'], label=f'Training {metric.capitalize()}', linewidth=2, marker='o')
        
        # Validation data
        ax.plot(epochs, history[f'val_{metric}'], color=colors['val'], label=f'Validation {metric.capitalize()}', linewidth=2, marker='o')
        
        # Add annotations for best/worst points
        best_epoch = np.argmax(history[f'val_{metric}']) if metric == 'accuracy' else np.argmin(history[f'val_{metric}'])
        best_value = history[f'val_{metric}'][best_epoch]
        ax.annotate(f'Best: {best_value:.4f}', xy=(best_epoch+1, best_value), xytext=(5, 5), 
                    textcoords='offset points', ha='left', va='bottom',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        ax.set_title(f'{metric.capitalize()} Over Epochs', fontsize=16)
        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel(metric.capitalize(), fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.7)
    
    # Learning Rate Over Epochs
    ax = fig.add_subplot(gs[2, :])
    if 'lr' in history:
        ax.plot(epochs, history['lr'], color=colors['lr'], linewidth=2, marker='o')
        ax.set_yscale('log')
        ax.set_title('Learning Rate Over Epochs', fontsize=16)
        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel('Learning Rate', fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Annotate initial and final learning rates
        ax.annotate(f'Initial LR: {history["lr"][0]:.2e}', xy=(1, history["lr"][0]), xytext=(5, 5), 
                    textcoords='offset points', ha='left', va='bottom',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        ax.annotate(f'Final LR: {history["lr"][-1]:.2e}', xy=(len(epochs), history["lr"][-1]), xytext=(5, 5), 
                    textcoords='offset points', ha='right', va='top',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    else:
        ax.text(0.5, 0.5, 'Learning Rate data not available', ha='center', va='center', fontsize=16)
    
    # Info text
    info_text = [f"Model: {model_id}",
                 f"Initial Learning Rate: {initial_lr}",
                 f"Final Learning Rate: {history['lr'][-1]:.2e}" if 'lr' in history else "Final Learning Rate: Not available",
                 f"Batch Size: {batch_size}",
                 f"Optimizer: {optimizer}",
                 f"Resolution: {resolution}",
                 f"Total Epochs: {len(history['accuracy'])}",
                 f"Early Stopping: {'Yes' if len(history['accuracy']) < 30 else 'No'}",
                 f"Train-Val Split: {100 - split:.2f}/{split:.2f}"]

    # Summary text
    summary_text = [f"Test Accuracy: {accuracy*100:.2f}%",
                    f"Best Val Accuracy: {max(history['val_accuracy'])*100:.2f}% (Epoch {np.argmax(history['val_accuracy'])+1})",
                    f"Lowest Val Loss: {min(history['val_loss']):.4f} (Epoch {np.argmin(history['val_loss'])+1})",
                    f"Overfitting: {'Not observed' if max(history['accuracy']) - max(history['val_accuracy']) < 0.05 else 'Observed'}"]

    # Add info and summary as text
    ax_info = fig.add_subplot(gs[3, 0])
    ax_info.axis('off')
    ax_info.text(0, 1, '\n'.join(info_text), verticalalignment='top', fontsize=12)
    ax_info.set_title('Model Information', fontsize=16)

    ax_summary = fig.add_subplot(gs[3, 1])
    ax_summary.axis('off')
    ax_summary.text(0, 1, '\n'.join(summary_text), verticalalignment='top', fontsize=12)
    ax_summary.set_title('Performance Summary', fontsize=16)

    # Overall title
    fig.suptitle(f"Training Metrics for MoviNet {model_id} Stream at {100 - split}%-{split}% train test split", fontsize=20, y=0.95)

    # Adjust layout and save
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f"training_metrics_for_{model_id}_at_train-test_split_of_{split}.png", dpi=300, bbox_inches='tight')
    plt.close()


In [27]:
def create_representative_dataset(generator, batch_size):
    """
    Create a representative dataset for TFLite integer quantization calibration.
    """
    def representative_data_gen():
        for _ in range(100):
            data, _ = next(generator())
            # Expand the batch dimension to match the expected input shape
            yield [data[None, ...]]
    return representative_data_gen

In [28]:
import json

def export_model_variants(model, model_id, resolution, accuracy, split, train_dict, model_weights):
    """
    Export the model in various formats for single-batch and multi-batch scenarios,
    as well as multi-frame scenarios.
    
    Args:
    model: The TensorFlow model to export
    model_id (str): Identifier for the model
    resolution (int): Input resolution for the model
    accuracy (float): Accuracy of the model
    """
    exported_models = {}
    split = 100 * split

    # Calculate initial model size and parameters
    initial_model_size = get_model_size(model_weights)
    initial_params = model.count_params()

    # Determine initial numerical operations
    initial_num_ops = get_numerical_operations(model)

    for batch_size, batch_type in zip([1, 2], ['single', 'dual']):
        for num_frames in [8]:
            saved_model_dir = f'saved_model_{batch_type}_batch_{num_frames}_frames'
            
            input_shape = [batch_size, num_frames, resolution, resolution, 3]
            
            export_saved_model.export_saved_model(
                model=model,
                input_shape=input_shape,
                export_path=saved_model_dir,
                causal=True,
                bundle_input_init_states_fn=False
            )
            
            # FP16 conversion
            converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.target_spec.supported_types = [tf.float16]
            tflite_model = converter.convert()
            
            fp16_filename = f'model_{model_id}_operations_using_fp16_with_{num_frames}_frames_at_{batch_type}_batch_from_{accuracy*100:.2f}%_model_at_training_split_{100-split}_{split}.tflite'
            with open(fp16_filename, 'wb') as f:
                f.write(tflite_model)
                
#             #INT8 conversion
#             converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
#             converter.optimizations = [tf.lite.Optimize.DEFAULT]
#             converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#             representative_dataset = create_representative_dataset(
#                 FrameGenerator(train_dict, num_frames, resolution, training=False),
#                 batch_size
#             )
#             converter.representative_dataset = representative_dataset

#             tflite_model = converter.convert()
            
#             int8_filename = f'model_{model_id}_operations_using_int8_with_{num_frames}_frames_at_{batch_type}_batch_from_{accuracy*100:.2f}%_model_at_training_split_{100-split}_{split}.tflite'
#             with open(int8_filename, 'wb') as f:
#                 f.write(tflite_model)
                
                                                          
            # Calculate exported model size
            exported_model_size = os.path.getsize(fp16_filename)
            
            exported_models[f'fp16_{batch_type}batch_{num_frames}frames'] = {
                'filename': fp16_filename,
                'input_shape': input_shape,
                'initial_model_size': initial_model_size,
                'exported_model_size': exported_model_size,
                'initial_num_ops': initial_num_ops,
                'exported_num_ops': 'fp16',
                'total_parameters': initial_params
            }
            
#             exported_models[f'int8_{batch_type}batch_{num_frames}frames'] = {
#                 'filename': int8_filename,
#                 'input_shape': input_shape,
#                 'initial_model_size': initial_model_size,
#                 'exported_model_size': os.path.getsize(int8_filename),
#                 'initial_num_ops': initial_num_ops,
#                 'exported_num_ops': 'int8',
#                 'total_parameters': initial_params
#             }

    # Save information to JSON file
    json_filename = f'model_{model_id}_export_info.json'
    with open(json_filename, 'w') as f:
        json.dump(exported_models, f, indent=4)

    return exported_models

def get_model_size(model_weights):
    
    size = os.path.getsize(model_weights)
    return size

def get_numerical_operations(model):
    # Determine the numerical operations used in the model
    if tf.keras.mixed_precision.global_policy().name == 'mixed_float16':
        return 'mixed_float16'
    else:
        return 'float32'

In [29]:
def plot_tflite_evaluation_results(results, label_names, model_id, resolution, split):
    plt.figure(figsize=(30, 40))
    split = 100 * split
    # Color palette
    colors = plt.cm.viridis(np.linspace(0, 1, 8))
    
    # 1. Accuracies bar plot
    plt.subplot(4, 2, (1, 2))
    model_types = list(results.keys())
    accuracies = [res['accuracy'] for res in results.values()]
    bars = plt.bar(model_types, accuracies, color=colors)
    plt.title(f'TFLite Model Accuracies (MoViNet-{model_id.upper()}, Resolution: {resolution})', fontsize=16)
    plt.ylabel('Accuracy', fontsize=14)
    plt.ylim(0, 1)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.4f}',
                 ha='center', va='bottom', fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 2. Input shapes and correct predictions
    plt.subplot(4, 2, 3)
    for i, (model_type, res) in enumerate(results.items()):
        input_shape = 'x'.join(map(str, res['input_shape']))
        correct = res['correct_predictions']
        total = res['total_predictions']
        plt.text(0, 1 - i / len(results), 
                 f"{model_type} (Shape: {input_shape}):\n{correct}/{total} correct",
                 fontsize=12, va='top')
    plt.axis('off')
    plt.title('Model Variants and Predictions', fontsize=16)
    
    # 3. Confusion matrix heatmap
    plt.subplot(4, 2, (4, 5))
    best_model = max(results, key=lambda x: results[x]['accuracy'])
    confusion_mat = results[best_model]['confusion_matrix']
    ax = plt.gca()
    sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='YlOrRd', xticklabels=label_names, yticklabels=label_names, cbar=True,ax=ax)
    plt.title(f'Confusion Matrix for Best Model ({best_model})', fontsize=16)
    plt.xlabel('Predicted Action', fontsize=14)
    plt.ylabel('True Action', fontsize=14)
    
    # Add interpretation guide as a legend
    guide_elements = [
        plt.Rectangle((0,0),1,1,fc="white", ec="gray", lw=2),
        plt.Rectangle((0,0),1,1,fc="lightgray", ec="gray", lw=2),
        plt.Rectangle((0,0),1,1,fc="darkgray", ec="gray", lw=2),
        Line2D([0], [0], color="black", lw=2)
    ]
    guide_labels = [
        "Correct predictions (diagonal)",
        "Minor misclassifications",
        "Major misclassifications",
        "Perfect classification line"
    ]
    ax.legend(guide_elements, guide_labels, loc='center left', bbox_to_anchor=(1, 0.5),
              title="Interpretation Guide", title_fontsize='large', fontsize='medium')
    
    # 4. Side-by-side comparison of TFLite accuracies
    plt.subplot(4, 2, 6)
    quantization_types = ['fp16']
    batch_types = ['single', 'dual', 'triple', 'quad']
    frame_counts = [4, 8, 12]
    
    x = np.arange(len(batch_types))
    width = 0.2
    
   
    for j, frame_count in enumerate(frame_counts):
        try:
            accuracies = [results[f'{quant_type}_{batch}batch_{frame_count}frames']['accuracy'] for batch in batch_types]
            plt.bar(x + (i*3 + j) * width, accuracies, width, label=f'{quant_type.upper()} ({frame_count} frames)', color=colors[i*3 + j])
        except KeyError as e:
            print(f"Missing data for {e}, skipping plot.")
            continue
    
    plt.xlabel('Batch Type', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.title('TFLite Accuracies: Quantization vs Batch Size vs Frame Count', fontsize=16)
    plt.xticks(x + width*3, batch_types, fontsize=12)
    plt.legend(title='Quantization & Frames', title_fontsize=12, fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 5. Latency comparison (if available)
    if 'latency' in next(iter(results.values())):
        plt.subplot(4, 2, 7)
        latencies = [res['latency'] for res in results.values()]
        plt.bar(model_types, latencies, color=colors)
        plt.title('Inference Latency Comparison', fontsize=16)
        plt.xlabel('Model Type', fontsize=14)
        plt.ylabel('Latency (ms)', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 6. Guide for interpreting the plots
    plt.subplot(4, 2, 8)
    plt.axis('off')
    plt.text(0, 1, "Guide for Interpreting the Plots:", fontsize=16, fontweight='bold')
    guide_text = """
    1. Accuracies Bar Plot: Shows the accuracy of each model variant. Higher bars indicate better performance.
    
    2. Model Variants and Predictions: Provides details on input shapes and correct predictions for each model.
    
    3. Confusion Matrix: Displays prediction errors for the best-performing model. Diagonal elements represent correct predictions.
    
    4. TFLite Accuracies Comparison: Compares accuracies across different quantization types, batch sizes, and frame counts.
    
    5. Latency Comparison (if available): Shows inference time for each model variant. Lower bars indicate faster inference.
    
    Key Observations:
    - Look for the highest accuracy in the bar plots.
    - Check if quantization (FP16) significantly affects accuracy.
    - Observe how different batch sizes and frame counts impact performance.
    - Consider the trade-off between accuracy and latency for deployment.
    """
    plt.text(0, 0.9, guide_text, fontsize=12, va='top', ha='left', wrap=True)
    
    plt.tight_layout()
    plt.savefig(f'evaluated_tflite_accuracies_MoviNet_{model_id.upper()}_{resolution}_from_model_with_training_split_{100-split}_{split}.png', dpi=600, bbox_inches='tight')
    plt.close()


In [30]:
import time

def get_top_k(probs, k=5, label_map=None):
    top_predictions = tf.argsort(probs, axis=-1, direction='DESCENDING')[:k]
    if label_map is not None:
        top_labels = tf.gather(label_map, top_predictions, axis=-1)
        top_labels = [label.decode('utf8') if isinstance(label, bytes) else label for label in top_labels.numpy()]
    else:
        top_labels = top_predictions.numpy().tolist()
    top_probs = tf.gather(probs, top_predictions, axis=-1).numpy()
    return tuple(zip(top_labels, top_probs))

def evaluate_exported_models(exported_models, test_ds, label_names, model_id, resolution, split):
    results = {}
    for model_type, model_data in exported_models.items():
        model_path = model_data['filename']
        input_shape = model_data['input_shape']
        batch_size, num_frames = input_shape[0], input_shape[1]
        print(f"Evaluating {model_type} model from path {model_path}...")
        try:
            interpreter = tf.lite.Interpreter(model_path=model_path)
            interpreter.allocate_tensors()
            runner = interpreter.get_signature_runner()
            init_states = {
                name: tf.zeros(x['shape'], dtype=x['dtype'])
                for name, x in runner.get_input_details().items()
                if name != 'image'
            }
            correct_predictions = 0
            total_predictions = 0
            latencies = []
            
            for frames, labels in test_ds:
                states = init_states.copy()
                
                # Handle different batch sizes
                actual_batch_size = min(batch_size, frames.shape[0])
                
                # Prepare input based on model's expected shape
                model_input = frames[:actual_batch_size, :num_frames]
                
                # Resize input if necessary to match expected shape
                if model_input.shape[2:] != input_shape[2:4]:
                    model_input = tf.image.resize(model_input, input_shape[2:4])
                
                # Ensure the input has the correct number of dimensions
                if len(model_input.shape) == 4:  # If it's (batch, height, width, channels)
                    model_input = tf.expand_dims(model_input, axis=1)  # Add time dimension
                
                # Reshape to match the expected input shape
                model_input = tf.reshape(model_input, [actual_batch_size, num_frames] + list(input_shape[2:]))
                
                # Timing the inference
                start_time = time.time()
                
                # Perform inference
                outputs = runner(**states, image=model_input)
                logits = outputs.pop('logits')
                #states = outputs  # Update states with the remaining outputs
                
                latency = (time.time() - start_time) * 1000  # Convert to milliseconds
                latencies.append(latency)
                
                # Process predictions
                probs = tf.nn.softmax(logits)
                for i in range(actual_batch_size):
                    top_k = get_top_k(probs[i], k=1, label_map=label_names)
                    predicted_label = top_k[0][0]
                    actual_label = label_names[labels[i].numpy()]
                    if isinstance(actual_label, bytes):
                        actual_label = actual_label.decode('utf8')
                    if predicted_label == actual_label:
                        correct_predictions += 1
                    total_predictions += 1
            
            # Calculate accuracy and latency statistics
            accuracy = correct_predictions / total_predictions
            avg_latency = np.mean(latencies)
            results[model_type] = {
                'accuracy': accuracy,
                'correct_predictions': correct_predictions,
                'total_predictions': total_predictions,
                'latency': avg_latency
            }
            print(f"Model {model_type}: Accuracy = {accuracy:.4f}, Avg Latency = {avg_latency:.2f} ms")
        
        except Exception as e:
            print(f"Error during evaluation of model {model_type}: {e}")
            results[model_type] = {
                'error': str(e),
                'accuracy': None,
                'latency': None
            }
    return results


In [31]:

def setup_and_train_movinet_models(models_dict):
    for model_id, params in models_dict.items():
        batch_size = params['batch_size']
        num_frames = params['num_frames']
        resolution = params['resolution']
        split = params['split']

        print(f"Setting up MoViNet-{model_id} with resolution {resolution}\n")
        
        train_dict, test_dict = train_test_split(t, test_split=split)
        per_model(t, train_dict, test_dict, model_id, split)
        
        train_generator = FrameGenerator(train_dict, num_frames, resolution, training=True)
        test_generator = FrameGenerator(test_dict, num_frames, resolution, training=False)

        train_ds = prepare_dataset(train_generator, batch_size)
        test_ds = prepare_dataset(test_generator, batch_size)
        
        use_positional_encoding = model_id in {'A3', 'A4', 'A5'}

        backbone = movinet.Movinet(
            model_id=model_id.lower(),
            causal=True,
            conv_type='2plus1d',
            se_type='2plus3d',
            activation='hard_swish',
            gating_activation='hard_sigmoid',
            use_positional_encoding=use_positional_encoding,
            use_external_states=False
        )

        backbone.Trainable = False

        model = movinet_model.MovinetClassifier(
            backbone,
            num_classes=600,
            output_states=True
        )

        # Load pre-trained weights
        url = f"https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_{model_id.lower()}_stream.tar.gz"
        tar_filename = f"movinet_{model_id.lower()}_stream.tar.gz"
        checkpoint_dir = f"movinet_{model_id.lower()}_stream"

        os.system(f"wget {url} -O {tar_filename} -q")
        os.system(f"tar -xvf {tar_filename}")

        checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
        checkpoint = tf.train.Checkpoint(model=model)
        status = checkpoint.restore(checkpoint_path)
        status.assert_existing_objects_matched()
        print(f"\nLoaded weights for MoViNet-{model_id} from {checkpoint_path}")

        model = build_classifier(batch_size, num_frames, resolution, backbone, 8)
        
        loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        model.compile(loss=loss_obj, optimizer='adam', metrics=['accuracy'])
        lr_scheduler = CustomLearningRateScheduler(lr_schedule)

        callbacks = [
            CustomLearningRateLogger(),
            CustomModelCheckpoint(
                filepath="trained_model.weights.h5",
                save_weights_only=True,
                verbose=1
            ),
            CustomReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=10,
                min_lr=1e-10,
                verbose=1,
                cooldown=1,
                min_delta=1e-3
            ),
            CustomEarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),
            lr_scheduler,
            CustomTensorBoard(log_dir='./logs')
        ]

        results = model.fit(
            train_ds,
            validation_data=test_ds,
            epochs=30,
            callbacks=callbacks,
            verbose=1,
            class_weight=class_weights
        )

        # Evaluate the model
        evaluation_results = model.evaluate(test_ds,return_dict=True)
        accuracy = evaluation_results['accuracy']
    

        # Plot training metrics
        plot_training_metrics(results, model_id, 0.001, batch_size, 'adam', resolution, split, accuracy)
       
        # Get actual and predicted labels
        actual, predicted = get_actual_predicted_labels(test_ds, model)
        label_names = list(test_generator.class_ids_for_name.keys())
                
        plot_confusion_matrix(actual, predicted, label_names, 'test', model_id,accuracy,split)
        plot_confusion_matrix2(actual, predicted, label_names, 'test', model_id,accuracy,split)
    
        print(f'\nAccuracy: {accuracy:.4f}')     
        
        model_id = model_id.lower()
        use_positional_encoding = model_id in {'a3', 'a4', 'a5'}
        resolution = resolution

        # Create backbone and model.
        backbone = movinet.Movinet(
            model_id=model_id,
            causal=True,
            conv_type='2plus1d',
            se_type='2plus3d',
            activation='hard_swish',
            gating_activation='hard_sigmoid',
            use_positional_encoding=use_positional_encoding,
            use_external_states=True,
        )

        model = movinet_model.MovinetClassifier(
                backbone,
                num_classes=8,
                output_states=True)

        model.load_weights('/kaggle/working/trained_model.weights.h5')
        # Export model variants
        exported_models = export_model_variants(model, model_id, resolution,accuracy, split, train_dict, '/kaggle/working/trained_model.weights.h5')
        # Evaluate exported TFLite models
#         tflite_results = evaluate_exported_models(exported_models, test_ds, label_names, model_id, resolution, split)
#         plot_tflite_evaluation_results(tflite_results, label_names, model_id, resolution, split)
        
setup_and_train_movinet_models(models)

Setting up MoViNet-A3 with resolution 256

Train-Test Split Distribution plot saved for A3 with 25.0% split.


I0000 00:00:1732792116.056368      23 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


movinet_a3_stream/
movinet_a3_stream/ckpt-1.data-00000-of-00001
movinet_a3_stream/ckpt-1.index
movinet_a3_stream/checkpoint

Loaded weights for MoViNet-A3 from movinet_a3_stream/ckpt-1
Epoch 1/30


I0000 00:00:1732792253.859776   11142 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1732792257.519189   11143 service.cc:148] XLA service 0x7bb78e7c19a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732792257.519222   11143 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1732792257.700685   11143 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    257/Unknown - 556s 2s/step - loss: 0.9999 - accuracy: 0.7071
Epoch 1: saving model to trained_model.weights.h5
257/257 [==============================] - 716s 2s/step - loss: 0.9999 - accuracy: 0.7071 - val_loss: 0.5384 - val_accuracy: 0.8125 - lr: 0.0010 - learning_rate: 0.0010
Epoch 2/30
257/257 [==============================] - ETA: 0s - loss: 0.4282 - accuracy: 0.8782
Epoch 2: saving model to trained_model.weights.h5
257/257 [==============================] - 553s 2s/step - loss: 0.4282 - accuracy: 0.8782 - val_loss: 0.4076 - val_accuracy: 0.8692 - lr: 0.0010 - learning_rate: 0.0010
Epoch 3/30
257/257 [==============================] - ETA: 0s - loss: 0.2557 - accuracy: 0.9240
Epoch 3: saving model to trained_model.weights.h5
257/257 [==============================] - 551s 2s/step - loss: 0.2557 - accuracy: 0.9240 - val_loss: 0.4956 - val_accuracy: 0.8750 - lr: 0.0010 - learning_rate: 0.0010
Epoch 4/30
257/257 [==============================] - ETA: 0s - loss: 0.1950 - accurac

/tmp/ipykernel_23/3868072059.py:23: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.



1/1 [==============================] - 0s 250ms/step
Enhanced confusion matrix saved as confusion_matrix_A3_test_accuracy_98.4012.png
Classification report saved as classification_report_A3_test_at_training_split_75.0_25.0.csv

Accuracy: 0.9840


W0000 00:00:1732808100.352265      23 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1732808100.352319      23 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1732808103.020844      23 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
W0000 00:00:1732808461.656117      23 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1732808461.656162      23 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
